########### NOTA BENE #################

QUESTO E' IL FILE CHE COSTRUISCE I GRAFI PER INFOGRAPH! (Modificato al 12 Febbraio)

E' la copia di ValutazioneGrafi, solo che valutazione grafi ha modifiche fino a i primi di febbraio, in particolare in quel notebook sono riuscito a trovare il modo per creare i grafi per layer 3. Per creare i grafi di layer 1 (dove dovevo rimuovere le parole non viste in layer 3 per la produzione dei word embeddings) ho dovuto fare qualche truscio che è presente in questo file.

In definitiva in questo file ci sono le costruzioni dei grafi di LAYER 1 e LAYER 3 dei documenti di E3C!

NOTA BENE: IL 29 FEBBRAIO MI SONO ACCORTO CHE LA PROVENIENZA ERA FONDAMENTALE PER GLI EMBEDDINGS DI INFOGRAPH, IN PARTICOLARE NEL LAYER 3 CI SONO 8000 DOCUMENTI PROVENIENTI DA AGENZIA ITALIANA DEL FARMACO, QUESTI DOCUMENTI SONO INUTILI PER LE ANALISI DEI PAZIENTI QUINDI LE RIMUOVIAMO!! PER OGNI CELLA NUOVA (29 FEBBRAIO, IN ALTO A ESSA METTO "#29" PER DISTINGUERLA DALLE ALTRE!

NOTA BENE:

MODIFICHE DEL 29 MARZO PER CREARE DATA LOADER SOLO PER MIUR E ITALIAN JOURNAL OF MEDICINE (PER FARE IN MODO DI RUNNARE INFOGRAPH SU DOCUMENTI LUNGHI SIMILI)

In [1]:
### Nota bene 2 (modifiche di fine gennaio inizio febbraio): #########

# sto cercando ora di modificare i grafi per non essere solon grafi di linea ma anzi essere del tipo di text level
# gcn dove la matrice di adiacenza è sulla base delle parole adiacenti con una finestra "p"

# per valutare la bonta della produzione vado sul file ValutazioneGrafi.ipynb che analizza nel dettaglio i dataset di grafo creati
# cosicche sfrutto i file pickle e non devo rirunnare tutto il codice!

#Vedo CostruzioneGrafoPython per capire come sono stati creati i grafi!

In [1]:
import pickle
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [2]:
with open('corpusParoleInutiliStemming.pkl', 'rb') as file:
    # Carica i dati dal file
    corpus_filtrato = pickle.load(file)
#len(corpus_filtrato[0])

In [3]:
with open("my_vecs_GennaioStemming.p", "rb") as file:
    word2Vec_model = pickle.load(file)

In [4]:
vocabolario = set()

for documento in corpus_filtrato:
    for parola in documento:
        vocabolario.add(parola)

In [5]:
word_names = word2Vec_model.wv.index_to_key

word_vectors = [word2Vec_model.wv[word] for word in word_names]

corpus_filtrato_embedding = []
for doc in corpus_filtrato:
    doc_embedding = []
    for parola in doc:
        doc_embedding.append(word2Vec_model.wv[parola])
    corpus_filtrato_embedding.append(doc_embedding)
    
#corpus_filtrato[1]
#word2Vec_model.wv['giovan']
#corpus_filtrato_embedding[0]

In [6]:
#Controllo se qualche documento ha una ripetizione di parole
def sono_tutte_diverse(lista):
    set_parole = set(lista) 
    return len(set_parole) == len(lista)

In [7]:
sono_tutte_diverse(corpus_filtrato[0]) # NB osservo che ci sono (ovviamente ripetizioni di parole) dentro un documento!! Quindi 
#devo tenere conto di ciò quando costruisco la matrice di adiacenza del grafo!!

corpus_filtrato_embedding[1]

def sono_tutte_diverse(array):
    # Converte l'array in un array bidimensionale
    array_bidimensionale = np.array(array)

    # Controlla se tutte le liste interne sono diverse
    return len(array_bidimensionale) == len(set(map(tuple, array_bidimensionale)))
def conta_vettori_unici(array):
    # Converte l'array in un array bidimensionale
    array_bidimensionale = np.array(array)
    return len(set(map(tuple, array_bidimensionale)))
#Doc0 = np.array(corpus_filtrato_embedding[0])
#print(conta_vettori_unici(Doc0),len(set(corpus_filtrato[0]))
      #MOLTO BENE! HO OTTENUTO UN MODO PER TROVARE IL NUMERO DI PAROLE UNICHE DI UN DOCUMENTO
#def Toglie_Duplicati(array):
 #   Docunique = set(map(tuple, array))
  #  Docunique_array = np.array(list(Docunique))
   # return Docunique_array

In [9]:
#Doc0unique = set(map(tuple, Doc0))
#Doc0unique_array = np.array(list(Doc0unique))
#len(Doc0unique_array) ok torna
#conta_vettori_unici(Doc0)

In [8]:
def checkParolaNuova(parola, dizionario):
            if parola in dizionario:
                #indice = diz[parola]
                return False
            else:
                #dizionario[parola]=len(dizionario)
                return True

In [11]:
''''
testo = ['io','oggi','sono','mela','oggi','banana','leggere','coltello','penna']
diz = {}
for parola in testo:
    print(checkParolaNuova(parola,diz))
diz
if(checkParolaNuova('cane',diz))==True:
    print('ciao')
diz

#provo su un toy example la mia strategia:

dim = len(set(testo))
A = np.zeros((dim,dim), dtype=int)
A
A = Crea_Matrice_Adiacenza(testo)
A
''''

SyntaxError: EOL while scanning string literal (1760815357.py, line 18)

In [9]:
#VERSIONE DEFINITIVA PER CREARE MATRICE DI ADIACENZA DATO UN TESTO, PER TENER CONTO SE LA PAROLA ERA GIA USCITA, PER P = 1
#(DIREI CHE P = 2 NON LO FACCIAMO)
def Crea_Matrice_Adiacenza(testo):
    dim = len(set(testo))
    A = np.zeros((dim,dim), dtype=int)
    diz = {}
    for i,parola in enumerate(testo):
        if (checkParolaNuova(parola,diz)==True):

            diz[parola]=len(diz) #indiceNuovo = len(diz)
            indicei = diz[parola] 

        else:      
            indicei = diz[parola] 
        if i != 0:  
            if (checkParolaNuova(testo[i-1],diz)==True):
                diz[testo[i-1]]=len(diz)
                indicej = diz[testo[i-1]]
                A[indicei,indicej]=1

            else:
                indicej = diz[testo[i-1]]
                A[indicei,indicej]=1
        if i!= len(testo)-1:
            if (checkParolaNuova(testo[i+1],diz)==True):
                diz[testo[i+1]]=len(diz)
                indicej = diz[testo[i+1]]
                A[indicei,indicej]=1

            else:
                indicej = diz[testo[i+1]]
                A[indicei,indicej]=1
    return A


NB: questo codice funziona se gli passo il testo con le parole (corpus_filtrato), non posso farlo con corpus filtrato embedding!

In [ ]:
# Nel ciclo for, per creare i file di formato "Data", devo avere le informazioni sia su corpus_filtrato_embedding che
# corpus_filtrato! QUindi per ovviare a ciò creo due liste in due cicli for (per embedding e non embedding e poi le accorpo)



In [10]:
A_list =[]
for i,Doc in enumerate(corpus_filtrato):
    A = Crea_Matrice_Adiacenza(Doc)
    A_list.append(A)
            

In [11]:
#29
import json
from tqdm.notebook import tqdm
import os

# Definisci la cartella contenente i file
cartella = r"C:\Users\enduser\OneDrive - Politecnico di Milano\Ingegneria matematica\Tesi\ProveDiCodice\E3C-Corpus\data_collection\Italian\layer3"

# Lista dei testi estratti
lista_source3 = []

# Itera su tutti i file nella cartella
for filename in tqdm(os.listdir(cartella)):
    if filename.endswith(".json"):  # Assicurati che siano file JSON
        filepath = os.path.join(cartella, filename)
        with open(filepath, 'r', encoding='utf-8') as file:  # Specifica l'encoding come utf-8
            try:
                # Carica il contenuto del file JSON
                data = json.load(file)
                if "source" in data:
                    source = data["source"]
                    lista_source3.append(source)
            except Exception as e:
                pass

AIFA_indexes = [index for index, source in enumerate(lista_source3) if source == 'Agenzia Italiana del Farmaco']

AIFA_indexes = [index for index, source in enumerate(lista_source3) if source == 'Agenzia Italiana del Farmaco']



  0%|          | 0/10213 [00:00<?, ?it/s]

In [16]:
set(lista_source3)

{'Agenzia Italiana del Farmaco',
 'Clinical Management Issues',
 'GCND Giornale di Clinica Nefrologica e Dialisi',
 'Hematology Reports',
 'Italian Journal of Emergency Medicine',
 'Italian Journal of Medicine',
 'La Pediatria Medica e Chirurgica',
 'Miur',
 'Pediatric Reports (MDPI)',
 'Pediatric Reports (PAGEPress)',
 'Prof. Dr. Francescopaolo Mattioli',
 'Sapienza Università di Roma',
 'microbiologia medica'}

In [17]:
#29 marzo
AIFA_indexes = [index for index, source in enumerate(lista_source3) if source == 'Agenzia Italiana del Farmaco']

Miur_indexes = [index for index, source in enumerate(lista_source3) if source == 'Miur']

Italian_Journal_of_Medicine_indexes = [index for index, source in enumerate(lista_source3) if source == 'Italian Journal of Medicine']

# Faccio prima per MIUR poi ripeto il codice per Italian journals! quindi sto attento che il codice non va runnato sequenzialmente!!

In [39]:
#29 Marzo
A_list =[]
for i,Doc in enumerate(corpus_filtrato):
    if i in Miur_indexes:
        A = Crea_Matrice_Adiacenza(Doc)
        A_list.append(A)

In [28]:
#29 Marzo
A_list =[]
for i,Doc in enumerate(corpus_filtrato):
    if i in Italian_Journal_of_Medicine_indexes:
        A = Crea_Matrice_Adiacenza(Doc)
        A_list.append(A)

In [21]:
# NB non andava bene la funzione con set perchè toglieva l'ordinamento!!!!!!!

def Toglie_Duplicati(array):
    Docunique_list = []
    seen = set()

    for item in array:
        item_tuple = tuple(item)
        if item_tuple not in seen:
            seen.add(item_tuple)
            Docunique_list.append(item_tuple)

    Docunique_array = np.array(Docunique_list)
    return Docunique_array

In [ ]:
#Docprova = np.array([[1,2,3],[7,8,9],[11,3,16],[7,7,4],[7,8,9]])
#print(Toglie_Duplicati(Docprova))

In [30]:
x_list = []
for Doc in corpus_filtrato_embedding:
    DocNew = Toglie_Duplicati(Doc)
    DocNew = np.array(DocNew)
    x = torch.tensor(DocNew, dtype=torch.float)
    x_list.append(x)

In [42]:
#29
x_list = []
for i,Doc in enumerate(corpus_filtrato_embedding):
    if i in Miur_indexes:
        DocNew = Toglie_Duplicati(Doc)
        DocNew = np.array(DocNew)
        x = torch.tensor(DocNew, dtype=torch.float)
        x_list.append(x)

In [41]:
#29
x_list = []
for i,Doc in enumerate(corpus_filtrato_embedding):
    if i in Italian_Journal_of_Medicine_indexes:
        DocNew = Toglie_Duplicati(Doc)
        DocNew = np.array(DocNew)
        x = torch.tensor(DocNew, dtype=torch.float)
        x_list.append(x)

In [43]:
datasetGrafi = []
for i in range(len(A_list)):
    x = x_list[i]
    A = A_list[i]
    edge_index = torch.tensor(np.column_stack(np.where(A == 1)), dtype=torch.long)
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    datasetGrafi.append(data)

In [31]:
#29 (è lo stesso codice ma va beh)
datasetGrafi = []
for i in range(len(A_list)):
    x = x_list[i]
    A = A_list[i]
    edge_index = torch.tensor(np.column_stack(np.where(A == 1)), dtype=torch.long)
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    datasetGrafi.append(data)

In [38]:
for datasetgrafo in datasetGrafi:
    print(datasetgrafo)

Data(x=[114, 10], edge_index=[2, 262])
Data(x=[104, 10], edge_index=[2, 222])
Data(x=[96, 10], edge_index=[2, 234])
Data(x=[94, 10], edge_index=[2, 214])
Data(x=[121, 10], edge_index=[2, 270])
Data(x=[107, 10], edge_index=[2, 244])
Data(x=[69, 10], edge_index=[2, 150])
Data(x=[97, 10], edge_index=[2, 224])
Data(x=[99, 10], edge_index=[2, 228])
Data(x=[109, 10], edge_index=[2, 242])
Data(x=[91, 10], edge_index=[2, 206])
Data(x=[111, 10], edge_index=[2, 272])
Data(x=[86, 10], edge_index=[2, 204])
Data(x=[104, 10], edge_index=[2, 258])
Data(x=[122, 10], edge_index=[2, 280])
Data(x=[106, 10], edge_index=[2, 264])
Data(x=[104, 10], edge_index=[2, 254])
Data(x=[116, 10], edge_index=[2, 262])
Data(x=[120, 10], edge_index=[2, 264])
Data(x=[125, 10], edge_index=[2, 277])
Data(x=[125, 10], edge_index=[2, 286])
Data(x=[128, 10], edge_index=[2, 288])
Data(x=[111, 10], edge_index=[2, 272])
Data(x=[109, 10], edge_index=[2, 256])
Data(x=[91, 10], edge_index=[2, 224])
Data(x=[125, 10], edge_index=[2, 

In [44]:
for datasetgrafo in datasetGrafi:
    print(datasetgrafo)

Data(x=[12, 10], edge_index=[2, 22])
Data(x=[9, 10], edge_index=[2, 16])
Data(x=[14, 10], edge_index=[2, 28])
Data(x=[23, 10], edge_index=[2, 44])
Data(x=[9, 10], edge_index=[2, 16])
Data(x=[9, 10], edge_index=[2, 16])
Data(x=[11, 10], edge_index=[2, 20])
Data(x=[11, 10], edge_index=[2, 20])
Data(x=[10, 10], edge_index=[2, 18])
Data(x=[12, 10], edge_index=[2, 22])
Data(x=[19, 10], edge_index=[2, 36])
Data(x=[18, 10], edge_index=[2, 34])
Data(x=[16, 10], edge_index=[2, 30])
Data(x=[14, 10], edge_index=[2, 37])
Data(x=[27, 10], edge_index=[2, 52])
Data(x=[23, 10], edge_index=[2, 44])
Data(x=[25, 10], edge_index=[2, 50])
Data(x=[12, 10], edge_index=[2, 22])
Data(x=[12, 10], edge_index=[2, 22])
Data(x=[13, 10], edge_index=[2, 24])
Data(x=[23, 10], edge_index=[2, 44])
Data(x=[10, 10], edge_index=[2, 20])
Data(x=[16, 10], edge_index=[2, 32])
Data(x=[13, 10], edge_index=[2, 26])
Data(x=[13, 10], edge_index=[2, 29])
Data(x=[9, 10], edge_index=[2, 16])
Data(x=[18, 10], edge_index=[2, 40])
Data(

Osservo che le dimensioni ora sono simili per le matrici di adiacenza di miur e italian journal of medicine.

In [32]:
len(datasetGrafi)

534

In [25]:
len(Miur_indexes)

1439

In [33]:
len(Italian_Journal_of_Medicine_indexes)

534

In [ ]:
#print(datasetGrafi[10].x)
#print(datasetGrafi[10].edge_index)

# "text": "Un paziente di 72 anni con parkinsonismo presenta piedi edematosi di color rossastro, sensibili, molto dolorosi."
#word2Vec_model.wv['parkinson']
#word2Vec_model.wv['72']
#word2Vec_model.wv['pazient']

# Ok tutto torna!

In [34]:
data_load = DataLoader(datasetGrafi, batch_size=128)

In [21]:
with open('datasetGrafi_NO_ripetizioni.pkl', 'wb') as file:
    pickle.dump(datasetGrafi, file)

# Save data_load to a pickle file
with open('data_load_NO_ripetizioni.pkl', 'wb') as file:
    pickle.dump(data_load, file)


In [27]:
with open('datasetGrafi_NO_ripetizioniMiurlayer3.pkl', 'wb') as file:
    pickle.dump(datasetGrafi, file)

# Save data_load to a pickle file
with open('data_load_NO_ripetizioniMiurlayer3.pkl', 'wb') as file:
    pickle.dump(data_load, file)


In [35]:
with open('datasetGrafi_NO_ripetizioniItalianJournalofMedicinelayer3.pkl', 'wb') as file:
    pickle.dump(datasetGrafi, file)

# Save data_load to a pickle file
with open('data_load_NO_ripetizioniItalianJournalofMedicinelayer3.pkl', 'wb') as file:
    pickle.dump(data_load, file)


NB: queste sono le modifiche al 4 febbraio: non considero la finestra p ma solo p = 1 per semplicità, la grossa modifica fatta
        è che non ci sono ripetizioni di parole (nella matrice di adiacenza non si formano nuovi nodi se una parola era già comparsa)

# Stessa cosa per layer1 !!! (6 Febbraio)

In [39]:
import pickle
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [40]:
with open('corpusParoleInutiliStemming_layer1.pkl', 'rb') as file:
    # Carica i dati dal file
    corpus_filtrato1 = pickle.load(file)
#len(corpus_filtrato[0])

In [ ]:
# len(corpus_filtrato) = 86

In [41]:
with open("my_vecs_GennaioStemming.p", "rb") as file:
    word2Vec_model = pickle.load(file)

NB RIMUOVO DAL CORPUS DI LAYER 1 LE PAROLE CHE NON ERANO MAI APPARSE NEL LAYER 3!

In [32]:
''''
for doc in corpus_filtrato1:
    for parola in doc:
        
        # NB: VOCABOLARIO E'IL SET DI PAROLE APPARSE IN LAYER 3, LE RIMUOVO DA LAYER 1 PERCHE' NON POSSONO ESSERE
        # TRADOTTE DAL WORD EMBEDDINGS (ALLENATO SOLO SUL LAYER3)
        
        if parola not in word2Vec_model.wv or parola =='mikulicz': # non so perchè ma questa parola dava problemi
            doc.remove(parola)
''''

In [42]:
# siccome il codice appena sopra sembra non funzionare correttamente nella rimozione delle parole da corpus_filtrato1,
# ho chiesto a chat gpt
# di migliorare il codice, questo è quello che mi ha fornito
for doc in corpus_filtrato1:
    parole_valide = []
    for parola in doc:
        try:
            # Controlla se la parola è nel vocabolario word2vec
            word2Vec_model.wv[parola]
            parole_valide.append(parola)
        except KeyError:
            # Se la parola non è presente, passa alla prossima
            pass
    # Sostituisci il documento con la lista di parole valide
    doc.clear()
    doc.extend(parole_valide)

In [36]:
len(corpus_filtrato1[0])

306

In [37]:
len(set(corpus_filtrato1[0]))

222

In [43]:
word_names = word2Vec_model.wv.index_to_key

word_vectors = [word2Vec_model.wv[word] for word in word_names]

corpus_filtrato_embedding1 = []
#count = 0
for doc in corpus_filtrato1:
    #count+=1
    #print(count)
    doc_embedding = []
    for parola in doc:
      # NB: VOCABOLARIO E'IL SET DI PAROLE APPARSE IN LAYER 3, LE RIMUOVO DA LAYER 1 PERCHE' NON POSSONO ESSERE
        # TRADOTTE DAL WORD EMBEDDINGS (ALLENATO SOLO SUL LAYER3)
        doc_embedding.append(word2Vec_model.wv[parola])
    corpus_filtrato_embedding1.append(doc_embedding)
    
#corpus_filtrato[1]
#word2Vec_model.wv['giovan']
#corpus_filtrato_embedding[0]

In [44]:
A_list1 =[]
for i,Doc in enumerate(corpus_filtrato1):
    A = Crea_Matrice_Adiacenza(Doc)
    A_list1.append(A)

In [45]:
x_list1 = []
for Doc in corpus_filtrato_embedding1:
    DocNew = Toglie_Duplicati(Doc)
    DocNew = np.array(DocNew)
    x = torch.tensor(DocNew, dtype=torch.float)
    print(x)
    x_list1.append(x)

tensor([[-0.0763, -0.1796, -0.0823,  ...,  0.0179,  0.0241, -0.1459],
        [-4.6677, 12.8738,  2.4164,  ...,  0.6273,  6.2944,  0.6810],
        [-0.1837, -0.1168,  0.2579,  ..., -0.8366,  0.0815, -0.9079],
        ...,
        [ 0.4316,  1.8728, -0.6643,  ..., -0.1390, -1.1040, -0.6955],
        [ 1.0222,  0.5878, -1.8398,  ...,  2.6197, -1.1463, -0.1489],
        [-2.1466,  2.9638, -6.2391,  ...,  0.4585, -0.7599, -4.6133]])
tensor([[-2.4758, -0.4560,  1.2712,  ..., -3.2092,  2.2867, -3.1343],
        [-1.5383, -0.2936,  1.0667,  ..., -0.3304, -0.2867, -1.2965],
        [-2.5504,  4.8371, -0.5959,  ..., -2.9301,  0.5331, -4.1783],
        ...,
        [ 0.0224, -0.0254, -0.0640,  ..., -0.0900, -0.0286, -0.0692],
        [ 1.0698, -0.9283, -0.9854,  ..., -4.3912, -1.2646, -3.4633],
        [-0.1411, -0.1017, -0.7845,  ..., -0.2372, -0.0756, -0.3983]])
tensor([[ 2.6338,  3.2786, -0.5161,  ...,  2.7700,  1.0450, -3.0388],
        [-1.9699,  1.8987, -4.7107,  ...,  0.2390,  1.7625, -5

tensor([[-4.6677e+00,  1.2874e+01,  2.4164e+00, -5.8969e+00, -8.5980e-01,
          9.0666e+00,  4.6441e+00,  6.2735e-01,  6.2944e+00,  6.8104e-01],
        [-1.1081e+00, -1.9313e+00,  7.1234e-01, -2.6270e-01,  1.0102e+00,
          1.5781e+00,  1.1869e+00, -1.0923e+00,  1.1826e+00, -1.0555e+00],
        [-2.5504e+00,  4.8371e+00, -5.9590e-01, -1.4751e+01,  7.6412e+00,
          1.1011e+01,  1.0311e+01, -2.9301e+00,  5.3312e-01, -4.1783e+00],
        [-2.2952e+00, -1.3854e+00, -7.6270e+00, -1.8251e+00, -5.0344e+00,
          4.6651e+00,  6.4823e+00, -1.1738e+00, -3.4680e-01, -2.6241e+00],
        [-6.7825e+00, -5.5271e+00, -3.5298e+00, -5.1428e+00,  2.8881e+00,
         -7.6643e-01,  3.5315e-01,  3.6654e+00, -2.5891e-01, -9.4154e+00],
        [ 3.4220e+00, -1.1509e+00, -6.5958e+00, -5.8556e+00, -2.4283e+00,
         -1.4381e+00, -4.6139e+00,  3.8797e-01,  4.9466e+00, -2.7196e+00],
        [-4.3478e+00, -7.7981e+00, -4.5454e+00, -3.4617e+00,  2.2420e-01,
          2.7447e-01,  3.9086e-0

tensor([[-0.4660,  0.9919,  4.5237,  ..., -2.7162,  0.6298, -2.6184],
        [-2.6948, -4.7560,  2.4069,  ..., -5.2702, -1.1953, -9.7347],
        [-2.5504,  4.8371, -0.5959,  ..., -2.9301,  0.5331, -4.1783],
        ...,
        [ 2.8367, -3.9191, -3.1942,  ...,  1.1435, -3.9748, -2.1175],
        [ 1.0037, -4.4404, -4.3522,  ...,  2.1968,  2.9975, -2.5135],
        [ 3.7510, -6.4392, -8.2979,  ..., -2.4668,  3.0865, -1.3846]])
tensor([[ 7.0979e-01,  3.9775e+00, -5.2999e-01,  ..., -2.8291e+00,
         -4.9738e-01, -2.0827e+00],
        [ 1.7331e+00, -6.0426e+00, -2.9485e+00,  ...,  6.4064e-01,
         -1.2825e-03, -2.9816e+00],
        [ 5.3545e-01, -3.9035e-01,  1.0284e+00,  ..., -8.8937e-01,
          2.0306e+00,  1.0232e-01],
        ...,
        [-2.4964e+00,  5.5896e-01,  1.4968e+00,  ...,  3.7288e+00,
          8.0357e+00, -9.6324e+00],
        [ 2.8977e+00,  2.5518e+00, -1.1094e+00,  ..., -2.8622e+00,
         -4.0940e+00, -7.3713e+00],
        [-2.5964e-01, -1.4788e+00, -3.

In [54]:
datasetGrafi1 = []
for i in range(len(A_list1)):
    x = x_list1[i]
    A = A_list1[i]
    edge_index = torch.tensor(np.column_stack(np.where(A == 1)), dtype=torch.long)
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    datasetGrafi1.append(data)

In [47]:
data_load1 = DataLoader(datasetGrafi1, batch_size=128)

In [50]:
with open('datasetGrafi_NO_ripetizioni_layer1.pkl', 'wb') as file:
    pickle.dump(datasetGrafi1, file)

# Save data_load to a pickle file
with open('data_load_NO_ripetizioni_layer1.pkl', 'wb') as file:
    pickle.dump(data_load1, file)

In [55]:
len(datasetGrafi)+len(datasetGrafi1)

2215

In [56]:
datasetGrafi1.extend(datasetGrafi)

In [57]:
datasetGrafi1and3 = datasetGrafi1

In [58]:
len(datasetGrafi1and3)

2215

In [60]:
data_load1and3 = DataLoader(datasetGrafi1and3, batch_size=128)

In [59]:
with open('data_load_NO_ripetizioni_layer1and_layer3.pkl', 'wb') as file:
    pickle.dump(data_load1and3, file)

In [60]:
with open('datasetGrafi_NO_ripetizioni_layer1and_layer3.pkl', 'wb') as file:
    pickle.dump(datasetGrafi1and3, file)

In [61]:
#29
with open('data_load_NO_ripetizioni_layer1and_layer3_senza_AIFA.pkl', 'wb') as file:
    pickle.dump(data_load1and3, file)

In [62]:
#29
with open('datasetGrafi_NO_ripetizioni_layer1and_layer3_senza_AIFA.pkl', 'wb') as file:
    pickle.dump(datasetGrafi1and3, file)